<div style="border:2px solid black; padding:10px;">
    <h3>Nhóm 15</h3>
    <h1 style='color:red;text-align:center;font-size:30px'>Chủ đề: Phân tích về doanh thu phim<b></b></h1>
</div>


<h3>Danh sách thành viên</h3>
<li> Nguyễn Quốc Hưng - 21120464 </li>
<li> Đoàn Đức Hữu - 21120465 </li>
<li> Võ Minh Khuê - 21120486</li>
<li> Hồ Trọng Lễ - 21120494</li>


<h1 style='color:blue;text-align:center;font-size:30px'><b>Phần I - B. Khám phá - Tiền xử xử lý dữ liệu </b></h1>


# Import thư viện


In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import seaborn as sns

import os
import re
import statistics
import seaborn as sns
sns.set(rc={'axes.facecolor':'lightblue', 'figure.facecolor':'lightgray'})

In [2]:
import sys
sys.executable

'/home/khuevo098/miniconda3/envs/min_ds-env/bin/python'

### Đọc file 'boxoffice_1990_2022.csv' vào dataframe df


In [3]:
df = pd.read_csv(
    './boxoffice_crawler/boxoffice_crawler/boxoffice_1990_2022.csv', encoding='latin1')
df.head(5)

ParserError: Error tokenizing data. C error: Expected 11 fields in line 10960, saw 12


## Khám phá dữ liệu


### Dữ liệu có bao nhiêu dòng và bao nhiêu cột?

In [ ]:
num_rows, num_cols = df.shape
num_rows, num_cols

### Mỗi dòng có ý nghĩa gì? Có vấn đề các dòng có ý nghĩa khác nhau không?
- Dựa vào kết quả trên, mỗi dòng thể hiện thông tin chi tiết của các phim trong khoảng thời gian từ năm 1980 đến năm 2022 được thu thập từ trang web Box Office Mojo.
- Không có dòng nào có ý nghĩa khác so với các dòng còn lại.

### Dữ liệu có dòng nào bị lặp không?

In [ ]:
duplicated_rows = df[df.duplicated()]
duplicated_nums = duplicated_rows.shape
if duplicated_nums[0] > 0:
    print(f'Có dữ liệu lặp: {duplicated_nums[0]} dòng')
else:
    print('Không có dữ liệu bị lặp')

Như kết quả trả về cho ta thấy, trong tập dữ liệu này có 487 dòng bị lặp. Vậy phải xử lý các dòng dữ liệu lặp: 

In [ ]:
df = df.drop_duplicates()

duplicated_rows = df[df.duplicated()]
duplicated_nums = duplicated_rows.shape
if duplicated_nums[0] > 0:
    print(f'Có dữ liệu lặp: {duplicated_nums[0]} dòng')
else:
    print('Không có dữ liệu bị lặp')

### Ý nghĩa của các cột


In [ ]:
schema_data = pd.read_csv(
    "./boxoffice_crawler/boxoffice_crawler/MovieRevenue_schema.csv", encoding='utf-8')
pd.set_option('max_colwidth', 400)
schema_data


### Kiểu dữ liệu của các cột


In [ ]:
df.info()

## Tiền xử lý

1. **Chuyển đổi các cột chứa số thành kiểu dữ liệu số**: Các cột như `domestic_revenue`, `world_revenue`, `opening_revenue`, `opening_theaters`, `budget`, và `in_release` đều chứa dữ liệu số, nhưng hiện đang được lưu trữ dưới dạng object. Chúng ta sẽ loại bỏ các ký tự không phải số (như dấu `$` và dấu phẩy) và chuyển đổi các cột này thành kiểu dữ liệu số.

2. **Chuyển đổi cột `release_date` thành kiểu dữ liệu datetime**: Điều này sẽ hữu ích cho việc phân tích sau này.


In [ ]:
# Remove $ and , in the revenue and budget columns
for col in ['domestic_revenue', 'world_revenue', 'opening_revenue', 'budget']:
    df[col] = df[col].replace({'\$': '', ',': ''}, regex=True)

# Convert columns to numeric type
for col in ['domestic_revenue', 'world_revenue', 'opening_revenue', 'opening_theaters', 'budget', 'in_release']:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Convert release_date to datetime
df['release_date'] = pd.to_datetime(df['release_date'], format='%d/%m/%Y')

In [ ]:
df.head(5)

### Với mỗi cột có dữ liệu dạng số, các giá trị được phân bố như thế nào?


Với các cột có kiểu dữ liệu số, ta cần tính:

- Tỉ lệ % (từ 0 đến 100) các giá trị thiếu
- Giá trị min
- Giá trị lower quartile (phân vị 25)
- Giá trị median (phân vị 50)
- Giá trị upper quartile (phân vị 75)
- Giá trị max

Sau đó lưu kết quả vào DataFrame `num_col_info_df`, trong đó:

- Tên của các cột là tên của các cột số trong `df`
- Tên của các dòng là: "missing_ratio", "min", "lower_quartile", "median", "upper_quartile", "max"

Để dễ nhìn, tất cả các giá trị bạn đều làm tròn với 1 chữ số thập phân bằng phương thức `.round(1)`.


In [ ]:
# Get numeric columns
numeric_cols = df.select_dtypes(include=np.number).columns.tolist()

# Calculate statistics
missing_ratio = df[numeric_cols].isnull().mean() * 100
min_values = df[numeric_cols].min()
lower_quartile = df[numeric_cols].quantile(0.25)
median = df[numeric_cols].median()
upper_quartile = df[numeric_cols].quantile(0.75)
max_values = df[numeric_cols].max()

# Create DataFrame
num_col_info_df = pd.DataFrame({
    'missing_ratio': missing_ratio,
    'min': min_values,
    'lower_quartile': lower_quartile,
    'median': median,
    'upper_quartile': upper_quartile,
    'max': max_values
})

# Round to 1 decimal place
num_col_info_df = num_col_info_df.round(2)
pd.options.display.float_format = '{:10,.2f}'.format
num_col_info_df.T

### Với mỗi cột có dữ liệu không phải dạng số, các giá trị được phân bố như thế nào?

Với các cột "title","distributor","MPAA","genres" ta cần tính:

- Tỉ lệ % (từ 0 đến 100) các giá trị thiếu
- Số lượng các giá trị (các giá trị ở đây là các giá trị khác nhau và ta không xét giá trị thiếu): với cột mà ứng với câu hỏi dạng multichoice (ví dụ, cột "genres"), mỗi giá trị có thể chứa nhiều choice (các choice được phân tách bởi dấu phẩy)
- Tỉ lệ % (từ 0 đến 100) của mỗi giá trị được sort theo tỉ lệ % giảm dần (ta không xét giá trị thiếu, tỉ lệ là tỉ lệ so với số lượng các giá trị không thiếu)

Lưu kết quả vào DataFrame `cat_col_info_df`, trong đó:

- Tên của các cột là tên của các cột không phải số trong `survey_df`
- Tên của các dòng là: "missing_ratio", "num_values", "value_ratios"

Để dễ nhìn, tất cả các giá trị bạn đều làm tròn với 1 chữ số thập phân bằng phương thức `.round(1)`.


In [ ]:
# Define categorical columns
cat_cols = ["title", "distributor", "MPAA", "genres"]

# Initialize DataFrame to store results
cat_col_info_df = pd.DataFrame(
    index=["missing_ratio", "num_values", "value_ratios"], columns=cat_cols)

for col in cat_cols:
    # Calculate missing ratio
    missing_ratio = df[col].isnull().mean() * 100

    # Calculate number of unique values
    # Split values by comma and flatten the list
    values = [val for sublist in df[col].dropna().str.split(',').tolist()
              for val in sublist]
    num_values = len(set(values))

    # Calculate value ratios
    value_counts = pd.Series(values).value_counts(normalize=True) * 100
    value_ratios = value_counts.sort_values(ascending=False).to_dict()

    # Store results
    cat_col_info_df.at["missing_ratio", col] = missing_ratio
    cat_col_info_df.at["num_values", col] = num_values
    cat_col_info_df.at["value_ratios", col] = value_ratios

# Round to 1 decimal place
cat_col_info_df = cat_col_info_df.applymap(
    lambda x: round(x, 1) if isinstance(x, (int, float)) else x)

cat_col_info_df

### Xử lý dữ liệu bị thiếu


- Tỷ lệ thiếu dữ liệu của từng cột


In [ ]:
df.agg(lambda x: x.isna().mean()*100).sort_values(ascending=False)

In [ ]:
# Drop the 'opening_theaters' and 'budget' columns
df = df.drop(['opening_theaters', 'budget'], axis=1)

# Drop rows with missing values
df = df.dropna()

df.agg(lambda x: x.isna().mean()*100).sort_values(ascending=False)
df.info()
df.shape

## Trực quan hóa và thống kê dữ liệu


In [ ]:
# Separate numeric and categorical columns
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = df.select_dtypes(
    include=['object', 'bool', 'datetime64[ns]']).columns

# Plot boxplots for numeric columns
for col in numeric_cols:
    plt.figure(figsize=(8, 6))
    plt.title(f'Boxplot of {col}')
    sns.boxplot(x=df[col])
    plt.show()

# Print basic statistics for categorical columns
print("\nBasic Statistics for Categorical Columns:")
for col in categorical_cols:
    print(f'\nColumn: {col}')
    print(df[col].describe())

In [ ]:
df.to_csv('./data/processed/processed_data.csv', index=False)